<a target="_blank" href="https://colab.research.google.com/github/nzolman/dynamicsai-rl-tutorial/blob/main/template.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# Only validated for Python 3.10!
!pip install torch ray[rllib] gymnasium numpy matplotlib tqdm

zsh:1: no matches found: ray[rllib]


In [2]:
import numpy as np
import torch
from gymnasium import Env
from gymnasium.spaces import Box
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# Creating an Environment and Defining the Objective
Here, we create a simple environment to interact with. We'll be examing the single inverted pendulum given by the following equations:

$$
\begin{align}
    \dot{\theta} & = \omega \\
    \dot{\omega} &= \frac{g}{L} \sin(\theta) + u
\end{align}
$$

Where $u$ is the control input into the system and the unstable equilibrium is at $(x,y) = (0, 1)$ and defined to be when $\theta =0$.

[Gymnasium documentation](https://gymnasium.farama.org/content/basic_usage/#/tutorials/environment_creation) \
[Gymnasium Github](https://github.com/Farama-Foundation/Gymnasium/tree/main)

In [ ]:
class PendEnv(Env):
    ''' 
    Pendulum environment with dynamics
    theta'(t) = omega
    omega'(t) = -(g/L) sin(theta) + u
    
    Objective: control pendulum to be in the upright position (theta=0)
    '''
    def __init__(self, config=None):
        super().__init__()
        
        <...>
        <...>

        
    def reset(self, seed=None, options=None):
        '''Reset the environment'''
        
        <...>
        <...>
        <...>
        
        return self.state, info
    
    def get_reward(self, obs, action):
        '''Obtain Reward'''
        
        <...>
        
        return <...>
        
    def step(self, action):
        '''
        Update environment by executing action
        '''
        
        <...>
        
        return self.state, reward, terminated, truncated, info 


## Env Sanity Check
Now that we've coded up an environment, let's check that it works!

In [ ]:
<...>

### No Action
Great, the functions at least seem to be outputting what they're supposed to. Let's step through the environment with no action for lots of steps and make sure this looks like it's working properly.

In [ ]:
<...>

In [ ]:
plt.plot(observation_list[:,0], label = 'x')
plt.plot(observation_list[:,1], label = 'y')
plt.plot(theta_list,label = r'$\theta$' )
plt.plot(omega_list, label = r'$\omega$')

plt.legend()
plt.show()

### Random Action
Ok, great! That seems to meet expectations. We start with $\theta \approx 0$, it swings down past the discontinuity at $\theta = \pi$ and approaches $\theta \approx 0$ again. 

For one final sanity check, let's incorporate random sampling

In [ ]:
<...>

In [ ]:
plt.plot(observation_list[:,0], label = 'x')
plt.plot(observation_list[:,1], label = 'y')
plt.plot(theta_list,label = r'$\theta$' )
plt.plot(omega_list, label = r'$\omega$')

plt.legend()
plt.show()

## RLLib Check
One final thing that we can do is appeal to RLlib, which we'll use in just a little bit, to check that our environment meets all the necessary API standards. 

(In practice, this is really useful because RLlib and Gymnasium have both undergone different API changes recently.)

In [ ]:
<...>

# Training an Agent

## RLlib
We'll be using RLlib ("RL Library") because it's gaining a lot of popularity, has multi-agent RL support, and has _many_ implementations for different DRL algorithms. However, most importantly, RLlib makes it very easy transition between developing locally on your laptop and scaling up to multi-node HPC clusters or cloud infrastructures without you having to change your code very much. 

We'll be toying around with their implementation of "Proximal Policy Optimization" (PPO), which is a type of policy gradient algorithm. All you have to know for the purposes of this tutorial is that the output of the policy network is a continuous action distribution, $\pi(x)$. And we sample the distribution to get a control value: $u \sim \pi(x)$. 

In particular, this particular implementation parameterizes a diagonal gaussian distribution: $\pi(x) = N(\mu(x), \sigma(x))$, so the output is the policy network is the mean and standard deviation. 

[RLlib Documentation](https://docs.ray.io/en/latest/rllib/index.html)

[Proximal Policy Optimization (PPO)](https://docs.ray.io/en/latest/rllib/rllib-algorithms.html#ppo)


## Build the Config
RLlib is "config-driven", and there are a ton of ways to customize your training environment, and even tune your hyperparameters. But we'll just be scratching the surface ever-so-lightly.

In [ ]:
<...>

## Train the algorithm

In [ ]:
<...>

And we've done it! We've trained! You can officially say that you've trained a Deep Reinforcement Learning policy. The catch is that we've only done this for a single training iteration. But what does this mean? Let's inspect this results dictionary, `res`, that we've created.

In [ ]:
<...>

As you can see, `num_env_steps_trained=4000`, meaning that we've trained on 4000 steps. Since we set $dt=0.01$, this means that it's about 40 seconds worth of data. We could write a loop and train for many more iterations, but let's inspect this first!

## Evaluating Our first Trained Agent

In [ ]:
<...>

In [ ]:
plt.plot(theta_list, label = r'$\theta$' )
plt.plot(omega_list, label = r'$\omega$')
plt.plot(action_list, label = 'u')
plt.legend()
plt.show()

Yeah, we didn't do so hot, huh? We can train longer, but let's dig a bit deeper before that. We can actually access the policy directly

## Other Analysis: Value Function and Policy
Because this is low-dimensional, we can actually poke at this a bit more than we normally can. Let's make a mesh of the state-space and look at both the policy function (i.e. the control) and the value function (i.e. "how much reward do we expect to get in a certain state").

In [ ]:
<...>

In [ ]:
<...>

### Plotting

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(1,3, figsize=(15,5))

# This lets us define where the colorbars should go
dividers = [make_axes_locatable(ax) for ax in axes]
cax_list = [divider.append_axes('top', size='5%', pad=0.25) for divider in dividers]  

# Create main plots
im_0 = axes[0].scatter(theta_mesh, omega_mesh, s = 10, cmap = plt.cm.RdBu,
                       c =control_mean)

im_1 = axes[1].scatter(theta_mesh, omega_mesh, s = 10, cmap = plt.cm.RdBu,
                       c =control_std)

im_2 = axes[2].scatter(theta_mesh, omega_mesh, s = 10, cmap = plt.cm.RdBu,
                       c =vf_vals)

ims = [im_0, im_1, im_2]

# create colorbars
cb_list = []
for im, cax in zip(ims, cax_list):
    cb= fig.colorbar(im, cax=cax, orientation='horizontal')
    cb_list.append(cb)  

cb_list[0].set_label('Action Mean', labelpad=-50)
cb_list[1].set_label('Action Std', labelpad=-50)
cb_list[2].set_label('Value Function', labelpad=-50)  


for ax in axes:
    ax.set_xlabel(r'$\theta$')
    ax.set_ylabel(r'$\omega$')

fig.tight_layout()
plt.show()

Ok! We've got some interesting structure going on here! Note that we don't actually have that much variance in the functions. The policy is approximately zero mean with unit width. The value function, however, has a bit more rich structure, although it isn't entirely clear what it means yet. 

# Train for more iterations!

## Setup
Before we train longer, let's wrap up a couple of these functions.

In [ ]:
def collect_data(env, algo, seed=0, n_steps = 1000):
    
    <...>
    
    return observation_list, action_list, rew_list


def plot_meshes(policy, mesh, plot_log=False):
    
    <...>
    
    return fig, axes, cb_list

## Train
Now let's train for 10 iterations or so. We'll evaluate our performance on the same initial condition after every training iteration. Something extra we'll do here is keep a copy of the policy states so we can access them later.

In [ ]:
from copy import deepcopy

env = PendEnv()
observations = []
actions = []
rews = []
policy_list = []
n_train_iterations = 10

for i in tqdm(range(n_train_iterations)):
    
    # train the agent for one iteration
    <...>
    
    # keep a copy of the policy
    <...>
    
    # collect data from the current agent
    <...>


## Analyzing the Training
First thing's first, we want to figure out if our performance has improved! Let's plot the total reward per iteration

In [ ]:
<...>

plt.plot(tot_rews)

plt.title('Total Reward Progress')
plt.xlabel('Training Iteration')
plt.ylabel('Total Reward')
plt.show()

That seems promising! Let's plot the rolled out trajectories:

In [ ]:
colors = plt.cm.tab10.colors

<...>

Perfect! We can see that, indeed, near the equilibrium we can stablize this pendulum! Let's plot the policy and value functions.

In [ ]:
<...>

# You're an RL Practitioner Now!
Play around with the code! Some motivating problems from the pendulum:
- What happens if we train even longer?
- What if we start from an initial condition in one of these "uncertain" regions? 
- What happens if you change to Sparse Rewards? (+1 if we're less than a maximum angle, -1 otherwise?)
- Robustness: 
    - What if we add noise to the observations? 
    - What if we change the model parameters? (e.g. $L =3$) 